<a href="https://colab.research.google.com/github/NahyeonKang/classify-spending/blob/main/build_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0. 음식점
1. 카페
2. 식재료
3. 편의점
5. 뷰티
6. 패션
7. 의료
8. 여행
9. 공연/영화
10. 스포츠
11. 기타 쇼핑
12. 간편결제
13. 교육비
14. 교통비
15. 생활요금
16. 기타


In [1]:
# import pandas as pd

# data = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/data.xlsx", header=None)

In [2]:
# data['category'] = "0"
# data

In [3]:
# data.columns=['document', 'label']

In [4]:
# data_test = data.loc[213:]

In [5]:
# data_test.to_csv("data_test.csv")

# GPT4 API로 분류하기

In [6]:
# ! pip install openai

In [7]:
# import openai

# openai.api_key = ""

In [8]:
# for i in range(166, 664):
#   spending = data.loc[i]['spending']

#   messages = [
#       {"role":"system", "content": "Categorize a given <spending> into the following categories. \n0.restaurant\n1.cafe\n2.grocery\n3.convenience store\n4.cosmetics\n5.fashion\n6.healthcare\n7.travel\n8.shows/movies\n9.sports\n10.other shoppings\n11.payment service\n12.education\n13.transportation\n14.utility bills\n15.Others  "},
#       {"role":"user", "content":f"<spending>\n{spending}"}
#   ]

#   response = openai.ChatCompletion.create(
#       model="gpt-3.5-turbo",
#       messages=messages
#   )

#   answer = response['choices'][0]['message']['content']
#   data.loc[i]['category']=answer

In [9]:
# i

In [10]:
# data.to_excel("/content/drive/MyDrive/Colab Notebooks/GPTlabeleddata.xlsx")

# GPT4 답변 & 크롤링 데이터로 huggingface Dataset 만들기

In [11]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
! pip install datasets

In [13]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files={'train': "/content/drive/MyDrive/data/train_labler_data.csv"})

In [14]:
dataset

DatasetDict({
    train: Dataset({
        features: ['document', 'label'],
        num_rows: 3475
    })
})

# BERT 학습하기

In [15]:
!pip install transformers datasets evaluate

In [16]:
!pip install accelerate -U

In [17]:
import random
import os
import numpy as np
import torch

def set_seeds(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False # for faster training, but not deterministic

set_seeds(seed=42)

In [18]:
from huggingface_hub import notebook_login

notebook_login()

## num_labels 추가

In [19]:
# from transformers import AutoTokenizer, BertForSequenceClassification

# tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")
# model = BertForSequenceClassification.from_pretrained("klue/roberta-base", num_labels=16).to('cuda')

In [20]:
# from transformers import BertTokenizerFast, ElectraForSequenceClassification, BertForSequenceClassification

# # KoALBERT 모델과 토크나이저 불러오기
# tokenizer = BertTokenizerFast.from_pretrained("classla/xlm-roberta-base-multilingual-text-genre-classifier")
# model = BertForSequenceClassification.from_pretrained("classla/xlm-roberta-base-multilingual-text-genre-classifier", num_labels=16).to('cuda')

In [21]:
from transformers import BertTokenizerFast, BertModel, BertForSequenceClassification

tokenizer = BertTokenizerFast.from_pretrained("kykim/bert-kor-base")
model = BertForSequenceClassification.from_pretrained("kykim/bert-kor-base", num_labels=16).to('cuda')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at kykim/bert-kor-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
def preprocess_function(examples):
    return tokenizer(examples["document"], truncation=True)

In [23]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

In [24]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [25]:
import evaluate

accuracy = evaluate.load("accuracy")

In [26]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

## wandb로 로깅

In [27]:
# !pip install wandb

In [28]:
# # Import wandb
# import wandb

# # Login with your authentication key
# wandb.login()

# # setup wandb environment variables
# %env WANDB_ENTITY='jychung/ai-keepit'
# %env WANDB_PROJECT='classify-spending'


In [29]:
import torch
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="ai.keepit/classifier",
    learning_rate= 5e-6,
    lr_scheduler_type="cosine",
    per_device_train_batch_size=16,
    num_train_epochs=10,
    weight_decay= 0.01, #0.01,
    save_strategy="epoch",
    push_to_hub=True,
    # report_to="wandb",
    run_name="kykim/bert-kor-base-10epoch-5e6"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,1.359800
1000,0.560600
1500,0.402500
2000,0.354500


TrainOutput(global_step=2180, training_loss=0.6420242344567535, metrics={'train_runtime': 311.4545, 'train_samples_per_second': 111.573, 'train_steps_per_second': 6.999, 'total_flos': 196148250136896.0, 'train_loss': 0.6420242344567535, 'epoch': 10.0})

In [30]:
trainer.push_to_hub()

'https://huggingface.co/jiiyy/classifier/tree/main/'

# Inference

In [31]:
import datasets
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm
import pandas as pd

pipe = pipeline("text-classification", model="jiiyy/classifier", device=0)
# pipe = pipeline("text-classification", model="nahyeonkang/classifier", device=0)
test_df = pd.read_csv("/content/drive/MyDrive/data/data_test.csv")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [32]:
test_df

,document,label
0,포토이즘박스 연,0
1,김밥마는아저씨,0
2,피플PC방,0
3,까치네분식,0
4,3공장,0
...,...,...
446,금고ATM입금,0
447,가시버시갈비탕.찜,0
448,펀비어킹,0
449,한국철도공사,0


In [33]:
for i in range(451):
  answer = pipe(test_df.iloc[i]['document'])
  if answer[0]['score'] < 0.5:
    test_df.loc[i, 'label'] = 'LABEL_15'
  else:
    test_df.loc[i, 'label'] = answer[0]['label']
  print(answer)

[{'label': 'LABEL_8', 'score': 0.8128930330276489}]
[{'label': 'LABEL_0', 'score': 0.8609032034873962}]
[{'label': 'LABEL_8', 'score': 0.4347449541091919}]
[{'label': 'LABEL_0', 'score': 0.9173723459243774}]
[{'label': 'LABEL_5', 'score': 0.3209393620491028}]
[{'label': 'LABEL_9', 'score': 0.933922529220581}]
[{'label': 'LABEL_2', 'score': 0.8126774430274963}]
[{'label': 'LABEL_2', 'score': 0.9684849381446838}]
[{'label': 'LABEL_1', 'score': 0.7735187411308289}]
[{'label': 'LABEL_0', 'score': 0.9312385320663452}]
[{'label': 'LABEL_0', 'score': 0.4763464033603668}]
[{'label': 'LABEL_5', 'score': 0.8537284135818481}]


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


[{'label': 'LABEL_2', 'score': 0.8809918761253357}]
[{'label': 'LABEL_1', 'score': 0.8605237007141113}]
[{'label': 'LABEL_0', 'score': 0.7696712613105774}]
[{'label': 'LABEL_0', 'score': 0.8850798606872559}]
[{'label': 'LABEL_1', 'score': 0.8633562922477722}]
[{'label': 'LABEL_5', 'score': 0.5898410081863403}]
[{'label': 'LABEL_0', 'score': 0.9323258996009827}]
[{'label': 'LABEL_11', 'score': 0.3470021188259125}]
[{'label': 'LABEL_1', 'score': 0.6145680546760559}]
[{'label': 'LABEL_11', 'score': 0.5580711960792542}]
[{'label': 'LABEL_0', 'score': 0.9361125826835632}]
[{'label': 'LABEL_2', 'score': 0.744148313999176}]
[{'label': 'LABEL_2', 'score': 0.8262504935264587}]
[{'label': 'LABEL_12', 'score': 0.6575023531913757}]
[{'label': 'LABEL_5', 'score': 0.5555786490440369}]
[{'label': 'LABEL_0', 'score': 0.9102302193641663}]
[{'label': 'LABEL_5', 'score': 0.3394293487071991}]
[{'label': 'LABEL_0', 'score': 0.9285407662391663}]
[{'label': 'LABEL_0', 'score': 0.9095016121864319}]
[{'label':

In [34]:
test_df['label'].value_counts()

LABEL_15    106
LABEL_0      93
LABEL_5      55
LABEL_2      54
LABEL_1      49
LABEL_11     35
LABEL_12     16
LABEL_6      12
LABEL_8      11
LABEL_9       9
LABEL_4       8
LABEL_7       3
Name: label, dtype: int64

In [35]:
test_df[test_df['label'] == 'LABEL_1']

,document,label
8,할리스커피 대명,LABEL_1
13,아마스빈버블티,LABEL_1
16,카페글렌공덕점,LABEL_1
20,일월수목원 카페 데이지원,LABEL_1
38,윈스터디카페,LABEL_1
39,아이스크림에 빠지다 염창점,LABEL_1
46,순천만갈대카페,LABEL_1
48,달콤샵(세계과자 아이스크림),LABEL_1
50,메가엠지씨커피 강서등촌점,LABEL_1
60,카페 델카미노,LABEL_1


In [36]:
test_df.to_excel("/content/drive/MyDrive/Colab Notebooks/labeled.xlsx")